In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import os
os.chdir('D:\APDS\Project\Yelp\py workspace')
REVIEW_FILE_PATH = 'D:\\APDS\\Project\\Yelp\\dataset\\RestaurantData\\review.csv'

In [2]:
sc = SparkContext("local", "Yelp - Restaurant Reviews Preprocessing")

In [3]:
spark = SparkSession \
    .builder \
    .appName("local") \
    .getOrCreate()

df = spark.read.csv(
    REVIEW_FILE_PATH, header=True, mode="DROPMALFORMED", quote='"', escape='"', multiLine=True
)

In [4]:
type(df)

pyspark.sql.dataframe.DataFrame

In [5]:
df.head(5)

[Row(_c0='25', review_id='Ia-w-nR1FrlzsiuEiqqlbg', user_id='u0LXt3Uea_GidxRW1xcsfg', business_id='Eox_Qq74oaFZ-YjthpHhBw', stars='3', date='2011-07-18', text='Service is really slow here. I waited for over 25 mins just for takeout. Definitely not worth the wait. However, they seem to have live music Friday nites, so if you are looking for entertainment and food sit-in is probably a lot faster', useful='1', funny='1', cool='1'),
 Row(_c0='93', review_id='BF0ANB54sc_f-3_howQBCg', user_id='ssuXFjkH4neiBgwv-oN4IA', business_id='JlNeaOymdVbE6_bubqjohg', stars='1', date='2014-08-09', text="We always go to the chevo's in chandler which is delicious, the one in ahwatukee is different for some reason.  Ordered the chicken rolled tacos today there was a tiny lil piece of chicken in each one, so basically I had 3 rolled deep fried tortillas yuck! :( No flavor what so ever.  Also ordered carne asada taco the meat tasted old like it was cooked earlier and just thrown on the grill to get warm.  Very

In [5]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- cool: string (nullable = true)



In [6]:
df.show(2, truncate=True)

+---+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|_c0|           review_id|             user_id|         business_id|stars|      date|                text|useful|funny|cool|
+---+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|  0|vkVSCC7xljjrAI4UG...|bv2nCi5Qv5vroFiqK...|AEx2SYEUJmTxVVB18...|    5|2016-05-28|Super simple plac...|     0|    0|   0|
|  1|n6QzIUObkYshz4dz2...|bv2nCi5Qv5vroFiqK...|VR6GpWIda3SfvPC-l...|    5|2016-05-28|Small unassuming ...|     0|    0|   0|
+---+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
only showing top 2 rows



In [7]:
df.columns

['_c0',
 'review_id',
 'user_id',
 'business_id',
 'stars',
 'date',
 'text',
 'useful',
 'funny',
 'cool']

In [10]:
##df.describe().show()

DataFrame[summary: string, _c0: string, review_id: string, user_id: string, business_id: string, stars: string, date: string, text: string, useful: string, funny: string, cool: string]

In [9]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+---+---------+-------+-----------+-----+----+----+------+-----+----+
|_c0|review_id|user_id|business_id|stars|date|text|useful|funny|cool|
+---+---------+-------+-----------+-----+----+----+------+-----+----+
|  0|        0|      0|          0|    0|   0|   0|     0|    0|   0|
+---+---------+-------+-----------+-----+----+----+------+-----+----+



In [10]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+---------+-------+-----------+-----+----+----+------+-----+----+
|_c0|review_id|user_id|business_id|stars|date|text|useful|funny|cool|
+---+---------+-------+-----------+-----+----+----+------+-----+----+
|  0|        0|      0|          0|    0|   0|   0|     0|    0|   0|
+---+---------+-------+-----------+-----+----+----+------+-----+----+



In [22]:
##df = df.dropna()

In [8]:
df.select('stars').show(10)

+-----+
|stars|
+-----+
|    5|
|    5|
|    5|
|    4|
|    4|
|    4|
|    3|
|    4|
|    3|
|    3|
+-----+
only showing top 10 rows



In [9]:
df = df.filter(df.stars != 3)

In [10]:
df.select('stars').distinct().show()

+-----+
|stars|
+-----+
|    5|
|    1|
|    4|
|    2|
+-----+



In [11]:
df.groupby('stars').count().show()

+-----+-------+
|stars|  count|
+-----+-------+
|    5|1197747|
|    1| 357190|
|    4| 870371|
|    2| 305846|
+-----+-------+



In [18]:
type(reviewcol)

pyspark.sql.dataframe.DataFrame

In [24]:
import re
import pandas as pd
import numpy as np
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import TextBlob
from textblob import Word
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import CountVectorizer

### Tokenization

In [13]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")

In [14]:
tokenized = tokenizer.transform(df)
regexTokenized = regexTokenizer.transform(df)

In [15]:
tokenized.select("text", "words").show(5)

+--------------------+--------------------+
|                text|               words|
+--------------------+--------------------+
|Super simple plac...|[super, simple, p...|
|Small unassuming ...|[small, unassumin...|
|Lester's is locat...|[lester's, is, lo...|
|Love coming here....|[love, coming, he...|
|Had their chocola...|[had, their, choc...|
+--------------------+--------------------+
only showing top 5 rows



In [16]:
regexTokenized.select("text", "words").show(5)

+--------------------+--------------------+
|                text|               words|
+--------------------+--------------------+
|Super simple plac...|[super, simple, p...|
|Small unassuming ...|[small, unassumin...|
|Lester's is locat...|[lester, s, is, l...|
|Love coming here....|[love, coming, he...|
|Had their chocola...|[had, their, choc...|
+--------------------+--------------------+
only showing top 5 rows



### Stopwords remover

In [19]:
dfWithWords = regexTokenized.select('text','words')
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
##show top 2 rows without truncation
remover.transform(dfWithWords).show(2, False) 

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [20]:
df.show(5)

+---+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|_c0|           review_id|             user_id|         business_id|stars|      date|                text|useful|funny|cool|
+---+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|  0|vkVSCC7xljjrAI4UG...|bv2nCi5Qv5vroFiqK...|AEx2SYEUJmTxVVB18...|    5|2016-05-28|Super simple plac...|     0|    0|   0|
|  1|n6QzIUObkYshz4dz2...|bv2nCi5Qv5vroFiqK...|VR6GpWIda3SfvPC-l...|    5|2016-05-28|Small unassuming ...|     0|    0|   0|
|  2|MV3CcKScW05u5LVfF...|bv2nCi5Qv5vroFiqK...|CKC0-MOWMqoeWf6s-...|    5|2016-05-28|Lester's is locat...|     0|    0|   0|
|  3|IXvOzsEMYtiJI0CAR...|bv2nCi5Qv5vroFiqK...|ACFtxLv8pGrrxMm6E...|    4|2016-05-28|Love coming here....|     0|    0|   0|
|  4|L_9BTb55X0GDtThi6...|bv2nCi5Qv5vroFiqK...|s2I_Ni76bjJNK9yG6...|    4|2016-05-28|Had their chocola...|     0|    0|   0|


In [28]:
dfWithoutStopWords = remover.transform(dfWithWords)
cv = CountVectorizer(inputCol="filtered", outputCol="features")
model = cv.fit(dfWithoutStopWords)

In [29]:
result = model.transform(dfWithoutStopWords)
result.show(5)

+--------------------+--------------------+--------------------+--------------------+
|                text|               words|            filtered|            features|
+--------------------+--------------------+--------------------+--------------------+
|Super simple plac...|[super, simple, p...|[super, simple, p...|(262144,[2,33,34,...|
|Small unassuming ...|[small, unassumin...|[small, unassumin...|(262144,[2,10,11,...|
|Lester's is locat...|[lester, s, is, l...|[lester, located,...|(262144,[10,18,21...|
|Love coming here....|[love, coming, he...|[love, coming, ye...|(262144,[0,1,2,17...|
|Had their chocola...|[had, their, choc...|[chocolate, almon...|(262144,[34,65,13...|
+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [32]:
len(model.vocabulary)

262144